## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

### Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_parquet("data/yellow_tripdata_2023-01.parquet")
df_valid = pd.read_parquet("data/yellow_tripdata_2023-02.parquet")

print(f"Answer to question #1 is: {len(df.columns)} columns")

Answer to question #1 is: 19 columns


### Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [3]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
print(f"Answer to question #2 is: {round(df['duration'].std(), 2)}")

Answer to question #2 is: 42.59


### Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [4]:
df['is_outlier'] = np.where((df['duration'] >= 60) | (df['duration'] <= 1), 1, 0)
perc_of_recs_remained = round(100 - (100 * df[df['is_outlier'] == 1].shape[0] / df.shape[0]))

df = df[df['is_outlier'] == 0]

print(f"Answer to question #3 is: {perc_of_recs_remained}%")

Answer to question #3 is: 98%


### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [5]:
df['PULocationID'] = df['PULocationID'].astype('str')
df['DOLocationID'] = df['DOLocationID'].astype('str')

df_to_ohe = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

v = DictVectorizer()
X = v.fit_transform(df_to_ohe)

print(f"Answer to question #4 is: {X.shape[1]} columns")

Answer to question #4 is: 515 columns


### Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [6]:
y = df['duration']

reg = LinearRegression().fit(X, y)
y_pred = reg.predict(X)

rms = mean_squared_error(y, y_pred, squared=False)
print(f"Answer to question #5 is: {round(rms, 2)}")

Answer to question #5 is: 7.65


### Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [7]:
df_valid['PULocationID'] = df_valid['PULocationID'].astype('str')
df_valid['DOLocationID'] = df_valid['DOLocationID'].astype('str')
df_valid['duration'] = (df_valid['tpep_dropoff_datetime'] - df_valid['tpep_pickup_datetime']).dt.total_seconds() / 60
df_valid = df_valid[(df_valid.duration >= 1) & (df_valid.duration <= 60)]

df_valid_to_ohe = df_valid[['PULocationID', 'DOLocationID']].to_dict(orient='records')

X_valid = v.transform(df_valid_to_ohe)
y_valid = df_valid['duration']

y_valid_pred = reg.predict(X_valid)

rms_valid = mean_squared_error(y_valid, y_valid_pred, squared=False)
print(f"Answer to question #5 is: {round(rms_valid, 2)}")

Answer to question #5 is: 7.81
